# Bar data

In [2]:
from ib_insync import *
util.startLoop()  

ib = IB()
ib.client.setConnectOptions('+PACEAPI')
ib.connect('127.0.0.1', 7497, clientId=17)

<IB connected to 127.0.0.1:7497 clientId=17>

In [3]:
import numpy as np
import pandas as pd
import yahoo_fin.stock_info as si
import datetime as dt
import os

In [4]:
def GetInputs(inputs_path):
    df = pd.read_csv(inputs_path, header= None)
    df_dict = dict(zip(df[0], df[1]))
    df_dict['symbols'] = df_dict['symbols'].split("|")
    return df_dict


def CreateFolderIfFolderDoesntExist(symbol, data_path, symbol_kind):
    folder_path = data_path + "/" + symbol_kind + "/" + symbol
    folder_exist = os.path.exists(folder_path)
    if not folder_exist:
        os.mkdir(folder_path)
        print("created folder for ", folder_path)


def GetDurationOfDataNeeded(file_path):
    df = pd.read_csv(file_path)
    df['date_time'] = pd.to_datetime(df['date'])
    most_recent_date = df['date_time'].max()
    now = dt.datetime.now()
    return (now - most_recent_date)

def GetTimeIncrement(interval_time):
    if interval_time == "10 mins":
        return dt.timedelta(seconds = 600)
    elif interval_time == "1 hour":
        return dt.timedelta(seconds = 3600)
    elif interval_time == "1 day":
        return dt.timedelta(days = 1)


def UpdatePriceFiles(symbol, data_path, symbol_kind, interval_time_list):
    contract = Stock(symbol, 'SMART', 'USD')
    ib.qualifyContracts(contract)
    for interval_time in interval_time_list:
        time_increment = GetTimeIncrement(interval_time)
        interval_time_string = interval_time.replace(" ", "_")
        file_path = data_path + "/" + symbol_kind + "/" + symbol + "/" + interval_time_string + ".csv"
        file_exist = os.path.exists(file_path)
        duration_time = '365 D'
        if file_exist:
            time_since_last_data_stored = GetDurationOfDataNeeded(file_path)
            if time_since_last_data_stored > time_increment:
                duration_time = str(time_since_last_data_stored.days + 1) + " D"
                bars = ib.reqHistoricalData(
                        contract,
                        endDateTime='',
                        durationStr=duration_time,
                        barSizeSetting=interval_time,
                        whatToShow='TRADES',
                        useRTH=True,
                        formatDate=1)
                existing_data_df = pd.read_csv(file_path)
                existing_data_df = existing_data_df.drop(existing_data_df.filter(regex='Unnamed').columns, axis=1)
                new_data_df = util.df(bars)
                merged_df = pd.concat([existing_data_df,new_data_df]).drop_duplicates().reset_index(drop=True)
                merged_df = merged_df.drop_duplicates(subset='date', keep="last")
                merged_df = merged_df.dropna()
                merged_df.to_csv(file_path)
                print("Added data to ", symbol, interval_time)
        else:
            bars = ib.reqHistoricalData(
                    contract,
                    endDateTime='',
                    durationStr=duration_time,
                    barSizeSetting=interval_time,
                    whatToShow='TRADES',
                    useRTH=True,
                    formatDate=1)
            df = util.df(bars)
            df.to_csv(file_path)
            print("Created new file for ", symbol, interval_time)
    return

def UpdateEarningsFile(symbol, symbol_kind):
    file_path = data_path + "/" + symbol_kind + "/" + symbol + "/earnings.csv"
    file_exist = os.path.exists(file_path)
    if file_exist:
        last_earnings = pd.to_datetime(pd.read_csv(file_path)['date']).max()
        now = dt.datetime.now()
        time_diff = now - last_earnings
        if time_diff.days > 60:
            earnings_dict = si.get_earnings_history(symbol)
            df_earnings = pd.DataFrame(earnings_dict)
            earnings_dates_list = pd.to_datetime(df_earnings.startdatetime).dt.date.to_list()
            df = pd.DataFrame({"date" :earnings_dates_list})
            df.to_csv(file_path)
            print("Updated earnings file for ", symbol)
    else:
        earnings_dict = si.get_earnings_history(symbol)
        df_earnings = pd.DataFrame(earnings_dict)
        earnings_dates_list = pd.to_datetime(df_earnings.startdatetime).dt.date.to_list()
        earnings_dates_list = [earning_date for earning_date in earnings_dates_list if earning_date <= dt.date.today()]
        df = pd.DataFrame({"date" :earnings_dates_list})
        df.to_csv(file_path)
        print("Created new file for earnings for", symbol)
    return


def UpdateData(symbol, data_path, symbol_kind, interval_time_list):
    CreateFolderIfFolderDoesntExist(symbol, data_path, symbol_kind)
    UpdatePriceFiles(symbol, data_path, symbol_kind, interval_time_list)
    UpdateEarningsFile(symbol, symbol_kind)
    return






In [6]:
inputs_path = "./../../Inputs/historical_data_inputs.csv"
data_path = "./../../HistoricalData"
interval_time_list = ['1 day', '1 hour', '10 mins']
inputs_dict = GetInputs(inputs_path)
for symbol in inputs_dict['symbols']:
    UpdateData(symbol, data_path, "stocks", interval_time_list)

Added data to  AAPL 1 day
Added data to  AAPL 1 hour
Added data to  AAPL 10 mins


In [102]:
UpdateData("MSFT", data_path, "stocks", ['10 mins'])

Added data to  MSFT 10 mins


In [117]:
contract = Stock('AAPL', 'SMART', 'USD')
duration_time = '2 D'
interval_time = '10 mins'
bars = ib.reqHistoricalData(
                        contract,
                        endDateTime='',
                        durationStr=duration_time,
                        barSizeSetting=interval_time,
                        whatToShow= 'OPTION_IMPLIED_VOLATILITY',
                        useRTH=True,
                        formatDate=1)

In [118]:
symbol

'OXY'

In [119]:
bars

[BarData(date=datetime.datetime(2022, 8, 2, 9, 30), open=0.26669173, high=0.28256624, low=0.26669173, close=0.28256624, volume=0.01, average=0.2826932360846294, barCount=0),
 BarData(date=datetime.datetime(2022, 8, 2, 9, 40), open=0.28256624, high=0.28256624, low=0.27780389, close=0.27939134, volume=0.01, average=0.2826932360846294, barCount=0),
 BarData(date=datetime.datetime(2022, 8, 2, 9, 50), open=0.27939134, high=0.28415369, low=0.27939134, close=0.28097879, volume=0.01, average=0.2838362006510093, barCount=0),
 BarData(date=datetime.datetime(2022, 8, 2, 10, 0), open=0.28097879, high=0.28415369, low=0.27939134, close=0.28097879, volume=0.01, average=0.2837727026195437, barCount=0),
 BarData(date=datetime.datetime(2022, 8, 2, 10, 10), open=0.28097879, high=0.28415369, low=0.28097879, close=0.28097879, volume=0.01, average=0.2837250790959446, barCount=0),
 BarData(date=datetime.datetime(2022, 8, 2, 10, 20), open=0.28097879, high=0.28097879, low=0.27780389, close=0.27780389, volume=0

In [120]:
a =Option('AAPL', '20220805', 130, 'C', "SMART", multiplier=100)

In [121]:
ib.qualifyContracts(a)

[Option(conId=569572336, symbol='AAPL', lastTradeDateOrContractMonth='20220805', strike=130.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  220805C00130000', tradingClass='AAPL')]

In [126]:
bars = ib.reqHistoricalData(
                        a,
                        endDateTime='',
                        durationStr=duration_time,
                        barSizeSetting=interval_time,
                        whatToShow= 'MIDPOINT',
                        useRTH=True,
                        formatDate=1)

In [127]:
bars

[BarData(date=datetime.datetime(2022, 8, 2, 9, 30), open=30.1, high=30.95, low=29.175, close=30.725, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2022, 8, 2, 9, 40), open=30.725, high=31.075, low=30.525, close=30.55, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2022, 8, 2, 9, 50), open=30.55, high=30.75, low=30.025, close=30.7, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2022, 8, 2, 10, 0), open=30.7, high=30.8, low=30.05, close=30.075, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2022, 8, 2, 10, 10), open=30.075, high=30.35, low=29.7, close=30.35, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2022, 8, 2, 10, 20), open=30.35, high=30.6, low=30.075, close=30.6, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2022, 8, 2, 10, 30), open=30.6, high=30.9, low=30.425, close=30.775, volume=-1.0, average=-1.0, barCount=-1),
 BarData